In [41]:
import pandas as pd
import numpy as np

In [42]:
data = pd.read_csv("./dataset/news.csv")

In [43]:
data.head()

,id,headline,text
0,uid-1,Market Advances 5.12 More,NEW YORK (AP) - A prime rate reduction by New ...
1,uid-2,District Boosts Request For Anti-Terrorism Aid...,Mayor Anthony A. Williams petitioned the White...
2,uid-3,"Election? Here's How You Do It, Mate.",From our downunder perspective here in Austral...
3,uid-4,The Biggest Boom Ever,We are about to rewrite history. Unless a rece...
4,uid-5,Economic Aide Sees Uptrend,"Sedate and scholarly Dr. Arthur Burns, the ex-..."


In [44]:
data.shape

(3000, 3)

In [45]:
# data.headline.value_counts()

In [46]:
data.isnull().sum()

id          0
headline    0
text        0
dtype: int64

In [47]:
# from sklearn.preprocessing import LabelEncoder

# le = LabelEncoder()

In [48]:
# data["headline"] = le.fit_transform(data["headline"])
# data.head()

In [49]:
# data.headline.nunique()

In [50]:
data["text_feature"] = data["headline"] + data["text"]

In [51]:
# data["text_feature"] = data["headline"]

In [52]:
import re
def filter_letters(raw_data):
    #data = re.sub("[^a-zA-Z]", ' ', raw_data)
    data = re.sub("[0-9]", ' ', raw_data)
    return data

In [53]:
data["text_feature"] = data["text_feature"].apply(filter_letters)

In [54]:
list = []
for txt in data.text_feature:
    list.append(len(txt))
    

In [55]:
max(list)

5050

In [56]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features= 5050, ngram_range= (1,2), stop_words = 'english', min_df= 1, max_df = 1.0,\
                                   norm = 'l2', sublinear_tf = True)

In [57]:
text_vec = vectorizer.fit_transform(data.text_feature)

In [58]:
indices = np.argsort(vectorizer.idf_)[::-1]

In [59]:
features_text = vectorizer.get_feature_names()
top_n = 10
top_features = [features_text[i] for i in indices[:top_n]]
print(top_features[0:10])

['tel', 'carbon', 'breast', 'fur', 'bcci', 'merck', 'surgical', 'rupee', 'welch', 'altman']


In [60]:
# np.savetxt('tf1.txt', text_vec.toarray(), delimiter=' ')

In [61]:
# text_vect_arr = text_vec.toarray()

In [62]:
# text_vect_arr[0:10]

In [63]:
# vectorizer.get_feature_names()

In [64]:
text_dataframe = pd.DataFrame(text_vec.toarray(), columns=vectorizer.get_feature_names())
text_dataframe.head()

,abandoned,abc,ability,able,abroad,absence,absolutely,abuse,academic,accelerate,...,york nov,york oct,york sept,york stock,york trading,young,younger,yuan,zero,zone
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
text_dataframe1 = text_dataframe.loc[:,top_features]

In [66]:
features = pd.concat([data, text_dataframe1], axis = 1)

In [67]:
features = features.drop(["text", "text_feature", "headline"], axis = 1)

In [68]:
features.shape

(3000, 11)

In [69]:
features = features.set_index("id")

In [70]:
features.head()

,tel,carbon,breast,fur,bcci,merck,surgical,rupee,welch,altman
id,,,,,,,,,,
uid-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uid-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uid-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uid-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
uid-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [71]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=1000).fit(features)

In [72]:
# print(pca.explained_variance_ratio_)
# print(pca.explained_variance_ratio_.cumsum())

In [73]:
# principal_features = pd.DataFrame(pca.transform(features))
# principal_features.head()

In [74]:
np.savetxt('tfm5.txt', features.as_matrix())

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [75]:
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score

In [76]:
for k in range(2,50,2):
    
    clusterer = GaussianMixture(n_components= k).fit(features)
    
    preds = clusterer.predict(features)
    
    print(silhouette_score(features, preds, random_state=0))

0.9851358075396173
0.9871469954600288
0.9888708363572859
0.9889343258303595
0.9918782065196902
0.9915166061355778
0.9935855050923613
0.9947822378215014
0.9945998934211524
0.9946617355622944
0.9948781619837955
0.9943765221074988
0.9933661137776353
0.9921700985801284
0.9909815344140495
0.9896666666666667
0.9896666666666667
0.9896666666666667
0.9896666666666667
0.9896666666666667
0.9896666666666667
0.9896666666666667
0.9896666666666667
0.9896666666666667


In [77]:
# from sklearn.cluster import KMeans
# score = {}
# for k in range(2,50,2):
#     clusterer = KMeans(n_clusters=k, random_state=100).fit(principal_features)
#     preds = clusterer.predict(principal_features)
#     score.update({k:silhouette_score(principal_features, preds, random_state=0)})

#     print(k,silhouette_score(principal_features, preds, random_state=0))

# print(score)

In [78]:
from sklearn.cluster import KMeans
score = {}
for k in range(2,50,2):
    clusterer = KMeans(n_clusters=k, random_state=100).fit(features)
    preds = clusterer.predict(features)
    score.update({k:silhouette_score(features, preds, random_state=0)})
#     sc = silhouette_score(features, preds, random_state=0)
#     score.append(sc)
    print(k,silhouette_score(features, preds, random_state=0))
#     features["cluster"] = preds
#     features["cluster"].to_csv("cluster_result.csv")
# print(score, np.argmax(score))
print(score)

2 0.9897345041057106
4 0.990273819084915
6 0.9920529923157322
8 0.9926594753039248
10 0.9928781284620585
12 0.9932604231819
14 0.9946390124075744
16 0.9950464319964925
18 0.9954179906860312
20 0.9951739179853797
22 0.9948781619837955
24 0.9942668529034987
26 0.9933347757820706
28 0.9921700985801284
30 0.9909815344140495
32 0.9896666666666667
34 0.9896666666666667
36 0.9896666666666667
38 0.9896666666666667
40 0.9896666666666667
42 0.9896666666666667
44 0.9896666666666667
46 0.9896666666666667
48 0.9896666666666667
{2: 0.9897345041057106, 4: 0.990273819084915, 6: 0.9920529923157322, 8: 0.9926594753039248, 10: 0.9928781284620585, 12: 0.9932604231819, 14: 0.9946390124075744, 16: 0.9950464319964925, 18: 0.9954179906860312, 20: 0.9951739179853797, 22: 0.9948781619837955, 24: 0.9942668529034987, 26: 0.9933347757820706, 28: 0.9921700985801284, 30: 0.9909815344140495, 32: 0.9896666666666667, 34: 0.9896666666666667, 36: 0.9896666666666667, 38: 0.9896666666666667, 40: 0.9896666666666667, 42: 0.9

In [79]:
sorted(score.items(), key=lambda x: x[1])

[(32, 0.9896666666666667),
 (34, 0.9896666666666667),
 (36, 0.9896666666666667),
 (38, 0.9896666666666667),
 (40, 0.9896666666666667),
 (42, 0.9896666666666667),
 (44, 0.9896666666666667),
 (46, 0.9896666666666667),
 (48, 0.9896666666666667),
 (2, 0.9897345041057106),
 (4, 0.990273819084915),
 (30, 0.9909815344140495),
 (6, 0.9920529923157322),
 (28, 0.9921700985801284),
 (8, 0.9926594753039248),
 (10, 0.9928781284620585),
 (12, 0.9932604231819),
 (26, 0.9933347757820706),
 (24, 0.9942668529034987),
 (14, 0.9946390124075744),
 (22, 0.9948781619837955),
 (16, 0.9950464319964925),
 (20, 0.9951739179853797),
 (18, 0.9954179906860312)]

In [80]:
from sklearn.cluster import KMeans
clusterer = KMeans(n_clusters=18, random_state=400).fit(features)
preds = clusterer.predict(features)
print(silhouette_score(features, preds, random_state=0))
features["cluster"] = preds
# features["cluster"].to_csv("cluster_result1.csv")

0.9954179906860312


In [81]:
features = features.reset_index()
features.head()

,id,tel,carbon,breast,fur,bcci,merck,surgical,rupee,welch,altman,cluster
0,uid-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,uid-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,uid-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,uid-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,uid-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [82]:
features.to_csv("cluster_result2.csv", columns = ["id", "cluster"], index= False)

In [83]:
features.head()

,id,tel,carbon,breast,fur,bcci,merck,surgical,rupee,welch,altman,cluster
0,uid-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,uid-2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,uid-3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,uid-4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,uid-5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
